<center>
<h4>CDS 110, Lecture 5</h4>
<font color=blue><h1>State Estimation for a Kinematic Car Model</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>

[Open in Google Colab](https://colab.research.google.com/drive/1SzTlvNUQIlOyQuNNu-SqQ8cFM3d1r2m-)

In this lecture, we will show how to construct an observer for a system in the presence of noise and distrubances.

Recall that an observer is a system that takes as input the (noisy) measured output of a system along with the applied input to the system, and produces as estimate $\hat x$ of the current state:

<center>
<img width=400 src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/estimation.png">
</center>



In [ ]:
# Import the various Python packages that we require
import numpy as np
import matplotlib.pyplot as plt
from math import pi, sin, cos, tan
try:
  import control as ct
  print("python-control", ct.__version__)
except ImportError:
  !pip install control
  import control as ct
import control.flatsys as fs

## White noise

A white noise process $W(t)$ is a signal that has the property that the mean of the signal is 0 and the value of the signal at any point in time $t$ is uncorrelated to the value of the signal at a point in time $s$, but that has a fixed amount of variance.  Mathematically, a white noise process $W
(t) \in \mathbb{R}^k$ satisfies

$$
\begin{aligned}
\mathbb{E}\{W(t)\} &= 0, &&\text{for all $t$} \\
\mathbb{E}\{W^\mathtt{T}(t) W(s)\} &= Q\, \delta(t-s) && \text{for all $s, t$},
\end{aligned}
$$

where $Q \in \mathbb{R}^{k \times k}$ is the "intensity" of the white noise process.

The python-control function `white_noise` can be used to create an instantiation of a white noise process:

In [ ]:
# Create the time vector that we want to use
Tf = 5
T = np.linspace(0, Tf, 1000)
dt = T[1] - T[0]

# Create a white noise signal
?ct.white_noise
Q = np.array([[0.1]])
W = ct.white_noise(T, Q)

plt.figure(figsize=[5, 3])
plt.plot(T, W[0])
plt.xlabel('Time [s]')
plt.ylabel('$V$');

To confirm this is a white noise signal, we can compute the correlation function

$$
\rho(\tau) = \mathbb{E}\{V^\mathtt{T}(t) V(t + \tau)\} = Q\, \delta(\tau),
$$

where $\delta(\tau)$ is the unit impulse function:

In [ ]:
# Correlation function for the input
tau, r_W = ct.correlation(T, W)

plt.plot(tau, r_W, 'r-')
plt.xlabel(r'$\tau$')
plt.ylabel(r'$r_W(\tau)$')

# Compute out the area under the peak
print("Signal covariance: ", Q.item())
print("Area under impulse: ", np.max(W) * dt)

## System definition: kinematic car

We make use of a simple model for a vehicle navigating in the plane, known as the "bicycle model".  The kinematics of this vehicle can be written in terms of the contact point $(x, y)$ and the angle $\theta$ of the vehicle with respect to the horizontal axis:

<table>
<tr>
    <td width="50%"><img src="https://fbswiki.org/wiki/images/5/52/Kincar.png" width=480></td>
    <td width="50%">
$$
\large\begin{aligned}
  \dot x &= \cos\theta\, v \\
  \dot y &= \sin\theta\, v \\
  \dot\theta &= \frac{v}{l} \tan \delta
\end{aligned}
$$
    </td>
</tr>
</table>

The input $v$ represents the velocity of the vehicle and the input $\delta$ represents the turning rate. The parameter $l$ is the wheelbase.

In [ ]:
# System definition
# Function to compute the RHS of the system dynamics
def kincar_update(t, x, u, params):
    # Get the parameters for the model
    l = params['wheelbase']             # vehicle wheelbase
    deltamax = params['maxsteer']         # max steering angle (rad)

    # Saturate the steering input
    delta = np.clip(u[1], -deltamax, deltamax)

    # Return the derivative of the state
    return np.array([
        np.cos(x[2]) * u[0],            # xdot = cos(theta) v
        np.sin(x[2]) * u[0],            # ydot = sin(theta) v
        (u[0] / l) * np.tan(delta)      # thdot = v/l tan(delta)
    ])

def kincar_output(t, x, u, params):
    return x                            # return x, y, theta (full state)

kincar_params={'wheelbase': 3, 'maxsteer': 0.5}

# Create nonlinear input/output system
kincar = ct.nlsys(
    kincar_update, kincar_output, name="kincar", params=kincar_params,
    inputs=('v', 'delta'), outputs=('x', 'y', 'theta'),
    states=('x', 'y', 'theta'))

In [ ]:
# Utility function to plot lane change manuever
def plot_lanechange(t, y, u, figure=None, yf=None, label=None):
    # Plot the xy trajectory
    plt.subplot(3, 1, 1, label='xy')
    plt.plot(y[0], y[1], label=label)
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")
    if yf is not None:
        plt.plot(yf[0], yf[1], 'ro')

    # Plot x and y as functions of time
    plt.subplot(3, 2, 3, label='x')
    plt.plot(t, y[0])
    plt.ylabel("$x$ [m]")

    plt.subplot(3, 2, 4, label='y')
    plt.plot(t, y[1])
    plt.ylabel("$y$ [m]")

    # Plot the inputs as a function of time
    plt.subplot(3, 2, 5, label='v')
    plt.plot(t, u[0])
    plt.xlabel("Time $t$ [sec]")
    plt.ylabel("$v$ [m/s]")

    plt.subplot(3, 2, 6, label='delta')
    plt.plot(t, u[1])
    plt.xlabel("Time $t$ [sec]")
    plt.ylabel("$\\delta$ [rad]")

    plt.subplot(3, 1, 1)
    plt.title("Lane change manuever")
    if label:
        plt.legend()
    plt.tight_layout()

We next define a desired trajectory for the vehicle.  For simplicity, we use a piecewise linear trajectory and then stabilize the system around that trajectory.  We will learn in a later lecture how to do this is in more rigorous way.  For now, it is enough to know that this generates a feasible trajectory for the vehicle.

In [ ]:
# Generate a trajectory for the vehicle
# Define the endpoints of the trajectory
x0 = np.array([0., -4., 0.]); u0 = np.array([10., 0.])
xf = np.array([40., 4., 0.]); uf = np.array([10., 0.])
Tf = 4
Ts = Tf / 100

# First 0.6 seconds: drive straight
T1 = np.linspace(0, 0.6, 15, endpoint=False)
x1 = np.array([6, -4, 0])
xd1 = np.array([x0 + (x1 - x0) * (t - T1[0]) / (T1[-1] - T1[0]) for t in T1]).transpose()

# Next 2.8 seconds: change to the other lane
T2 = np.linspace(0.6, 3.4, 70, endpoint=False)
x2 = np.array([35, 4, 0])
xd2 = np.array([x1 + (x2 - x1) * (t - T2[0]) / (T2[-1] - T2[0]) for t in T2]).transpose()

# Final 0.6 seconds: drive straight
T3 = np.linspace(3.4, Tf, 15, endpoint=False)
xd3 = np.array([x2 + (xf - x2) * (t - T3[0]) / (T3[-1] - T3[0]) for t in T3]).transpose()

T = np.hstack([T1, T2, T3])
xr = np.hstack([xd1, xd2, xd3])
ur = np.array([u0 for t in T]).transpose()

# Now create a simple controller to stabilize the trajectory
P = kincar.linearize(x0, u0)
K, _, _ = ct.lqr(
    kincar.linearize(x0, u0),
    np.diag([10, 100, 1]), np.diag([10, 10])
)

# Construct a closed loop controller for the system
ctrl, clsys = ct.create_statefbk_iosystem(kincar, K)
resp = ct.input_output_response(clsys, T, [xr, ur], x0)

xd = resp.states
ud = resp.outputs[kincar.nstates:]

plot_lanechange(T, xd, ud, label='feasible')
plot_lanechange(T, xr, ur, label='reference')

In [ ]:
# Simulation of the open loop trajectory
sys_resp = ct.input_output_response(kincar, T, ud, xd[:, 0])
plt.plot(sys_resp.states[0], sys_resp.states[1])
plt.axis([0, 40, -5, 5])
plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.gca().set_aspect('equal')

## State estimation

To illustrate how we can estimate the state of the trajectory, we construct an observer that takes the measured inputs and outputs to the system and computes an estimate of the state, using a estimator with dynamics

$$
\dot{\hat x} = f(\hat x, u) - L(C \hat x - y)
$$

Note that we go ahead and use the nonlinear dynamics for the prediction term, but the linearization for the correction term.

We can determine the estimator gain $L$ via multiple methods:
* Eigenvalue placement
* Optimal estimation (Kalman filter)

### Eigenvalue placement

In [ ]:
# Define the outputs to use for measurements
C = np.eye(2, 3)

# Compute the linearization of the nonlinear dynamics
P = kincar.linearize([0, 0, 0], [10, 0])

# Compute the gains via eigenvalue placement
L = ct.place(P.A.T, C.T, [-1, -2, -3]).T

# Estimator update law
def estimator_update(t, xhat, u, params):
    # Extract the inputs to the estimator
    y = u[0:2]     # first two system outputs
    u = u[2:4]     # inputs that were applied

    # Update the state estimate
    xhatdot = kincar.updfcn(t, xhat, u, kincar_params) \
      - params['L'] @ (C @ xhat - y)

    # Return the derivative
    return xhatdot

estimator = ct.nlsys(
    estimator_update, None, name='estimator',
    states=kincar.nstates, params={'L': L},
    inputs= kincar.state_labels[0:2] + kincar.input_labels,
    outputs=[f'xh{i}' for i in range(kincar.nstates)],
)
print(estimator)
print(estimator.params)

In [ ]:
# Run the estimator from a different initial condition
estresp = ct.input_output_response(
    estimator, T, [xd[0:2], ud], [0, -3, 0])

fig, axs = plt.subplots(3, 1, figsize=[5, 4])

axs[0].plot(estresp.time, estresp.outputs[0], 'b-', T, xd[0], 'r--')
axs[0].set_ylabel("$x$")
axs[0].legend([r"$\hat x$", "$x$"])

axs[1].plot(estresp.time, estresp.outputs[1], 'b-', T, xd[1], 'r--')
axs[1].set_ylabel("$y$")

axs[2].plot(estresp.time, estresp.outputs[2], 'b-', T, xd[2], 'r--')
axs[2].set_ylabel(r"$\theta$")
axs[2].set_xlabel("Time $t$ [s]")

plt.tight_layout()

### Kalman filter

An alternative mechanism for creating an estimator is through the use of optimal estimation (Kalman filtering).

Suppose that we have (very) noisy measurements of the system position, and also have disturbances taht are applied to our control signal.

In [ ]:
# Disturbance and noise covariances
Qv = np.diag([0.1**2, 0.01**2])
Qw = np.eye(2) * 0.1**2

u_noisy = ud + ct.white_noise(T, Qv)
sys_resp = ct.input_output_response(kincar, T, u_noisy, xd[:, 0])

# Create noisy version of the measurements
y_noisy = sys_resp.outputs[0:2] + ct.white_noise(T, Qw)

plt.plot(y_noisy[0], y_noisy[1], 'k-')
plt.plot(sys_resp.outputs[0], sys_resp.outputs[1], 'b-')
plt.axis([0, 40, -5, 5])
plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.legend(['measured', 'actual'])
plt.gca().set_aspect('equal')

A Kalman filter allows us to estimate the optimal state give measurements of the inputs and outputs, as well as knowldge of the covariance of the signals.

In [ ]:
# Compute the Kalman gains (linear quadratic estimator)
L_kf, _, _ = ct.lqe(P.A, P.B, C, Qv, Qw)

kfresp = ct.input_output_response(
    estimator, T, [y_noisy, ud], [0, -3, 0],
    params={'L': L_kf})

fig, axs = plt.subplots(3, 1, figsize=[5, 4])

axs[0].plot(T, y_noisy[0], 'k-')
axs[0].plot(kfresp.time, kfresp.outputs[0], 'b-', T, sys_resp.outputs[0], 'r--')
axs[0].set_ylabel("$x$")
axs[0].legend([r"$\hat x$", "$x$"])

axs[1].plot(T, y_noisy[1], 'k-')
axs[1].plot(kfresp.time, kfresp.outputs[1], 'b-', T, sys_resp.outputs[1], 'r--')
axs[1].set_ylabel("$y$")

axs[2].plot(kfresp.time, kfresp.outputs[2], 'b-', T, sys_resp.outputs[2], 'r--')
axs[2].set_ylabel(r"$\theta$")
axs[2].set_xlabel("Time $t$ [s]")

plt.tight_layout()

We can get a better view of the convergence by plotting the errors:

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=[5, 4])

axs[0].plot(kfresp.time, kfresp.outputs[0] - sys_resp.outputs[0])
axs[0].plot([T[0], T[-1]], [0, 0], 'k--')
axs[0].set_ylabel("$x$ error")
axs[0].set_ylim([-1, 1])

axs[1].plot(kfresp.time, kfresp.outputs[1] - sys_resp.outputs[1])
axs[1].plot([T[0], T[-1]], [0, 0], 'k--')
axs[1].set_ylabel("$y$ error")
axs[1].set_ylim([-1, 1])

axs[2].plot(kfresp.time, kfresp.outputs[2] - sys_resp.outputs[2])
axs[2].plot([T[0], T[-1]], [0, 0], 'k--')
axs[2].set_ylabel(r"$\theta$ error")
axs[2].set_xlabel("Time $t$ [s]")
axs[2].set_ylim([-0.2, 0.2])

plt.tight_layout()

## Output feedback control

We next construct a controller that makes use of the estimated state.  We will attempt to control the longitudinal position using the steering angle as an input, with the velocity set to the desired velocity (no tracking of the longitudinal position).

In [ ]:
# Compute the linearization of the nonlinear dynamics
P = kincar.linearize([0, 0, 0], [10, 0])

# Extract out the linearized dynamics from delta to y
Alat = P.A[1:3, 1:3]
Blat = P.B[1:3, 1:2]
Clat = P.C[1:2, 1:3]

sys = ct.ss(Alat, Blat, Clat, 0)
print(sys)

In [ ]:
# Construct a state space controller, using LQR
Qx = np.diag([1, 10])
Qu = np.diag([1])

K, _, _ = ct.lqr(Alat, Blat, Qx, Qu)
print(f"{K=}")

kf = -1 / (Clat @ np.linalg.inv(Alat - Blat @ K) @ Blat)
print(f"{kf=}")

### Direct state space feedback

We start by checking the response of the system assuming that we measure the state directly.



In [ ]:
# Construct a controller for the full system
def ctrl_output(t, x, u, params):
  r_v, r_y = u[0:2]
  x = u[3:5]                # y, theta
  return np.vstack([r_v, -K @ x + kf * r_y])
ctrl = ct.nlsys(
    None, ctrl_output, name='ctrl',
    inputs=['r_v', 'r_y', 'x', 'y', 'theta'],
    outputs=['v', 'delta']
)
print(ctrl)

In [ ]:
# Direct state feedback
clsys_direct = ct.interconnect(
    [kincar, ctrl],
    inputs=['r_v', 'r_y'],
    outputs=['x', 'y', 'theta', 'v', 'delta'],
)
print(clsys_direct)

In [ ]:
# Run a simulation
clresp_direct = ct.input_output_response(
    clsys_direct, T, [10, xd[1]], X0=[0, -3, 0])

plt.plot(clresp_direct.outputs[0], clresp_direct.outputs[1])
plt.plot(xd[0], xd[1], 'r--')
# plt.plot(clresp.time, clresp.outputs[1])
plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.gca().set_aspect('equal')

Note the "lag" in the $x$ coordinate. This comes from the fact that we did not use feedback to maintain the longitudinal position as a function of time, compared with the desired trajectory.  To see this, we can look at the commanded speed versus the desired speed:

In [ ]:
plot_lanechange(T, xd, ud, label="desired")
plot_lanechange(T, clresp_direct.outputs[0:2], clresp_direct.outputs[-2:], label="actual")

From this plot we can also see that there is a very large input $\delta$ applied at $t=0$.  This is something we would have to fix if we were to implement this on a physical system (-1 rad $\approx -60^\circ$!).

### Estimator-based control

We now consider the case were we cannot directly measure the state, but instead have to estimate the state from the commanded input and measured output.  We can insert the estimator into the system model by reconnecting the inputs and outputs.  The `ct.interconnect` function provides the needed flexibility:

In [ ]:
print("ct.interconnect(syslist, connections=None, inplist=None, outlist=None, params=None, check_unused=True, add_unused=False, ignore_inputs=None, ignore_outputs=None, warn_duplicate=None, debug=False, **kwargs)\n\t")
print(ct.interconnect.__doc__)

We now create the system model that includes the estimator (observer).  Here is the system we are trying to construct:

<img width=600 src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/ssctrl.png">
</center>

(Be careful with the notation: in the diagram above $y$ is the measured outputs, which for our system are the $x$ and $y$ position of the vehicle, so overusing the symbol $y$.)

In [ ]:
# Connect the system, estimator, and controller
clsys_estim = ct.interconnect(
    [kincar, estimator, ctrl],
    inplist=['ctrl.r_v', 'ctrl.r_y', 'estimator.x', 'estimator.y'],
    inputs=['r_v', 'r_y', 'noise_x', 'noise_y'],
    outlist=[
        'kincar.x', 'kincar.y', 'kincar.theta',
        'estimator.xh0', 'estimator.xh1', 'estimator.xh2',
        'ctrl.v', 'ctrl.delta'
    ],
    outputs=['x', 'y', 'theta', 'xhat', 'yhat', 'thhat', 'v', 'delta'],
    connections=[
        ['kincar.v', 'ctrl.v'],
        ['kincar.delta', 'ctrl.delta'],
        ['estimator.x', 'kincar.x'],
        ['estimator.y', 'kincar.y'],
        ['estimator.delta', 'ctrl.delta'],
        ['estimator.v', 'ctrl.v'],
        ['ctrl.x', 'estimator.xh0'],
        ['ctrl.y', 'estimator.xh1'],
        ['ctrl.theta', 'estimator.xh2'],
    ],
)
print(clsys_estim)

In [ ]:
# Run a simulation with no noise first
clresp_nonoise = ct.input_output_response(
    clsys_estim, T, [10, xd[1], 0, 0], X0=[0, -3, 0, 0, -5, 0])

plt.plot(clresp_nonoise.outputs[0], clresp_nonoise.outputs[1])
plt.plot(xd[0], xd[1], 'r--')

plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.gca().set_aspect('equal')

In [ ]:
# Add some noise
Qv = np.diag([0.1**2, 0.01**2])
Qw = np.eye(2) * 0.1**2

u_noise = ct.white_noise(T, Qv)
y_noise = ct.white_noise(T, Qw)

# Run a simulation
clresp_noisy = ct.input_output_response(
    clsys_estim, T, [10, xd[1], y_noise], X0=[0, -3, 0, 0, -5, 0])

plt.plot(clresp_direct.outputs[0], clresp_direct.outputs[1], label='direct')
plt.plot(clresp_nonoise.outputs[0], clresp_nonoise.outputs[1], label='nonoise')
plt.plot(clresp_noisy.outputs[0], clresp_noisy.outputs[1], label='noisy')
plt.legend()
plt.plot(xd[0], xd[1], 'r--')

plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.gca().set_aspect('equal')

In [ ]:
# Plot the differences in y to make differences more clear
plt.plot(
    clresp_nonoise.time, clresp_nonoise.outputs[1] - clresp_direct.outputs[1],
    label='nonoise')
plt.plot(
    clresp_noisy.time, clresp_noisy.outputs[1] - clresp_direct.outputs[1],
    label='noisy')
plt.legend()
plt.plot([clresp_nonoise.time[0], clresp_nonoise.time[-1]], [0, 0], 'r--')

plt.xlabel("Time [s]")
plt.ylabel("$y$ [m]");

In [ ]:
# Show the control inputs as well as the final trajectory
plot_lanechange(T, xd, ud, label="desired")
plot_lanechange(T, clresp_noisy.outputs[0:2], clresp_noisy.outputs[-2:], label="actual")

### Things to try

* Wrap a controller around the velocity (or $x$ position) in addition to the lateral ($y$) position
* Change the amounts of noise in the sensor signal
* Add disturbances to the dynamics (corresponding to wind, hills, etc)